In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import RegularGridInterpolator

from bounded_rand_walkers.cpp import (
    bound_map,
    funky,
    generate_data,
    get_binned_2D,
    get_binned_data,
    get_cached_filename,
)
from bounded_rand_walkers.rad_interp import exact_radii_interp, inv_exact_radii_interp
from bounded_rand_walkers.relief_matrix_shaper import gen_shaper2D
from bounded_rand_walkers.rotation_steps import get_pdf_transform_shaper
from bounded_rand_walkers.shaper_general import gen_rad_shaper_exact, shaper_map
from bounded_rand_walkers.utils import (
    approx_edges,
    cache_dir,
    get_centres,
    match_ref,
    normalise,
)

Path("plots").mkdir(exist_ok=True)

In [ ]:
# Generate a single set of data.

pdf_kwargs = dict(width=2.0)


def get_f_i(r):
    """Calculate f_i for given radii."""
    return np.array([funky([c, 0], **pdf_kwargs) for c in r])


bound_name = "circle"
n_bins = 300

data_kwargs = dict(
    cache_dir=cache_dir,
    samples=int(1e7),
    seed=np.arange(10),
    blocks=2,
    bound_name=bound_name,
    **pdf_kwargs
)

In [ ]:
filenames = get_cached_filename(squeeze=False, **data_kwargs)
if not all(p.is_file() for p in filenames):
    generate_data(squeeze=False, max_workers=2, cache_only=True, **data_kwargs)

(
    g_x_edges,
    g_y_edges,
    g_x_centres,
    g_y_centres,
    f_t_x_edges,
    f_t_y_edges,
    f_t_x_centres,
    f_t_y_centres,
    f_t_r_edges,
    f_t_r_centres,
    g_numerical,
    f_t_numerical,
    f_t_r_numerical,
) = get_binned_data(filenames=filenames, n_bins=n_bins, max_step_length=2)

f_i_r_analytical = get_f_i(f_t_r_centres)

In [ ]:
# 1D shaper calculation.
r_shaper = gen_rad_shaper_exact(f_t_r_centres, vertices=bound_name)

# Select valid elements.
valid_r = ~np.isclose(r_shaper, 0)

# 1D reconstruction of the intrinsic pdf as a function of step length only.
f_i_r_num = f_t_r_numerical[valid_r] / r_shaper[valid_r]

f_i_r_num_radii = f_t_r_centres[valid_r].copy()

# 1D analytical transformed distribution.
f_t_r_analytical = f_i_r_analytical * r_shaper

# Analytical transformed.
f_i_r_analytical_trans = f_t_r_analytical[valid_r] / r_shaper[valid_r]

In [ ]:
# Plot f_t and f_i.
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

# Plot f_t.
ax = axes[0]

ax.plot(
    f_t_r_centres,
    normalise(f_t_r_edges, f_t_r_analytical * f_t_r_centres),
    label="Analytical",
    zorder=1,
)
ax.plot(
    f_t_r_centres,
    normalise(f_t_r_edges, f_t_r_numerical),
    label="Numerical",
    zorder=2,
    linestyle="--",
    c="C1",
)

# Plot f_i.
ax = axes[1]

f_i_r_analytical_trans_norm = normalise(
    f_i_r_num_radii, f_i_r_analytical_trans * f_i_r_num_radii
)

ax.plot(
    f_i_r_num_radii,
    f_i_r_analytical_trans_norm,
    label="Transformed Analytical",
    zorder=1,
    linestyle="-.",
    c="C0",
)

ax.plot(
    f_t_r_centres,
    match_ref(
        x=f_t_r_centres,
        y=f_i_r_analytical * f_t_r_centres,
        ref_x=f_i_r_num_radii,
        ref_y=f_i_r_analytical_trans_norm,
    ),
    label="Analytical",
    zorder=2,
    linestyle="--",
    c="C2",
)

ax.plot(
    f_i_r_num_radii,
    normalise(f_i_r_num_radii, f_i_r_num),
    label="Numerical",
    zorder=1,
    c="C1",
    linestyle="--",
)

# Labels.
axes[0].set_xlabel(r"$\ell$")
axes[0].set_ylabel(r"$\tilde{f}_t(\ell)$")

axes[1].set_xlabel(r"$\ell$")
axes[1].set_ylabel(r"$\tilde{f}_i(\ell)$")

for ax, title in zip(axes, ["(a)", "(b)"]):
    ax.legend(loc="best")
    ax.grid(linestyle="--", alpha=0.4)
    ax.text(0, 1.04, title, transform=ax.transAxes, fontsize=11)

# Move the two subplots closer to each other.
fig.tight_layout()

# Finally, save into the 'plots' directory.
fig.savefig(
    (Path("plots") / "circle_reconstruction").with_suffix(".png"), bbox_inches="tight"
)